<h1>Portfolio Optimisation</h1>
In this project, we will try to implement the conditional portfolio optimisation method for capital allocation. The paper does not explicit state the implementation of this, so we will try to prototype it with python using machine learning methods. The model will be given a list of tickers which are highly probable to be profitable, and then we use conditional portfolio optimisation to optimise the capital allocation according to current market sentiment.

Part 1, we will first try to find the tickers with highest probability of profit by implementing the method outlined in the paper in the Journal of Forecasting.

Part 2, we will use previously collected tickers to form a optimised allocation of capital. In order for this to work, essentially we plug in macroeconomic data to represent the general market sentiment and also the allocation of tickers which will be generated as a grid, then we get a predicted output which will be the thing we are trying to maximise, in this example we will try to maximise 60 day sharpe ratio, annualised return (365D) <-> alpha, also the weighted sum of sharpe ratio and return (want to account for both), and we just rank the output and use the combination with the highest objective value. However, we note that the complexity of the problem scales exponentially which means that we will need to be selective in terms of the grid search when searching for optimal allocations.

<h1>Part 1: Stock selection with machine learning</h1>
As outlined in the paper, we will select stock tickers using several different machine learning models and also an ensemble model (which had shown to produced best returns). For simplicity sake, we will only be using stocks listed on NASDAQ (5044 tickers)

In [10]:
import pandas as pd
import requests
import numpy as np
import json

# Download stock tickers listed on NASDAQ (use if NASDAQ have new listings)
"""
EXCHANGE_CODE="NASDAQ"
url = f"https://eodhd.com/api/exchange-symbol-list/{EXCHANGE_CODE}?api_token=667822cc36e777.79338265&fmt=json"
response = requests.get(url)
ticker_data = pd.DataFrame(response.json())
ticker_data.to_csv("tickers.csv", index=False)
ticker_data
"""

# Load stock tickers
ticker_data = pd.read_csv("tickers.csv")

# Load into list
ticker_list = list(ticker_data["Code"] + ".US") # Easier to plug into EOD api call
ticker_list


['AACG.US',
 'AACI.US',
 'AACIU.US',
 'AACIW.US',
 'AADI.US',
 'AADR.US',
 'AAGR.US',
 'AAGRW.US',
 'AAIDX.US',
 'AAL.US',
 'AAME.US',
 'AAOI.US',
 'AAON.US',
 'AAPB.US',
 'AAPD.US',
 'AAPL.US',
 'AAPU.US',
 'AAXJ.US',
 'ABAT.US',
 'ABCB.US',
 'ABCL.US',
 'ABCS.US',
 'ABEO.US',
 'ABIO.US',
 'ABL.US',
 'ABLLL.US',
 'ABLLW.US',
 'ABLV.US',
 'ABLVW.US',
 'ABNB.US',
 'ABOS.US',
 'ABSI.US',
 'ABTS.US',
 'ABUS.US',
 'ABVC.US',
 'ABVE.US',
 'ABVEW.US',
 'ABVX.US',
 'ACAB.US',
 'ACABU.US',
 'ACAC.US',
 'ACACU.US',
 'ACACW.US',
 'ACAD.US',
 'ACAHW.US',
 'ACB.US',
 'ACCD.US',
 'ACDC.US',
 'ACET.US',
 'ACGL.US',
 'ACGLN.US',
 'ACGLO.US',
 'ACHC.US',
 'ACHL.US',
 'ACHV.US',
 'ACIC.US',
 'ACIU.US',
 'ACIW.US',
 'ACLS.US',
 'ACLX.US',
 'ACMR.US',
 'ACNB.US',
 'ACNT.US',
 'ACON.US',
 'ACONW.US',
 'ACRS.US',
 'ACRV.US',
 'ACST.US',
 'ACT.US',
 'ACTG.US',
 'ACTU.US',
 'ACVA.US',
 'ACWI.US',
 'ACWX.US',
 'ACXP.US',
 'ADAG.US',
 'ADAP.US',
 'ADBE.US',
 'ADC-PA.US',
 'ADD.US',
 'ADEA.US',
 'ADI.US',
 'ADI